In [ ]:
from tqdm.auto import tqdm
import csv
import pandas as pd

## GPT Sample Generation

In [4]:
from openai import OpenAI
import time
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

OPEN_AI_KEY = os.environ.get('OPEN_AI_KEY')
client = OpenAI(api_key=OPEN_AI_KEY)

In [7]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a system that can generate chat prompts that understands values in Chinese communication. This includes indirect communication, which relies on subtlety and hints and context, as well as showing respect in front of elders, as well as humility. "},
    {"role": "system", "content": "Here are 3 examples: Input:\nCONTEXT: You are a caregiver in a nursing home, conversing with an elderly Chinese resident, Mrs. Li, in English. You notice she hasn't finished her food from yesterday.\n Mrs. Li: Hello!\n Response:\n Hello Mrs. Li. Was the food from yesterday to your liking?\n \n Input:\n CONTEXT: You are a hotel receptionist, speaking to an elderly Chinese couple that is checking out. They have not paid yet, but are insisting they have. \n Woman: We would like to check out, we paid yesterday.\n Response:\n I apologize if there's been a misunderstanding. I will double check our system again.\n \n Input:\n CONTEXT: You are conversing with a Chinese coworker to provide feedback on their work, which is a little lacking. \n Coworker: Hello!\n Response:\n Hello, how are you? I've reviewed your work. It's quite comprehensive. Would you like to go over this section though?"},
    {"role": "user", "content": "Give me 40 chat prompts of unique scenarios that exhibit values in Chinese communication, especially the values of indirectness and respect to elders. Use the format of the examples given with the context."}
  ]
)

In [ ]:
output = completion.choices[0].message.content

In [11]:
# Write samples to data file (appending)
with open('data/gpt_samples.txt', 'a') as f:
    f.write(output)

## Parse Data 
txt to csv

In [4]:
def parse_sample_files(file):
    f = open(file, "r")
    is_question = False
    prompts = []
    ideal = []
    for x in f: 
        x = x.strip()
        if "CONTEXT: " in x:
            i = x.index("CONTEXT: ")
            cur_prompt = x[i+9:]
            is_question = True

        elif is_question:
            cur_prompt += "\n" + x
            is_question = False
            prompts.append(cur_prompt)
        
        elif "Response: " in x:
            resp = x.strip().replace("Response: ","")
            ideal.append(resp)
    f.close()
    return prompts, ideal

In [79]:
nh_prompts, nh_ideal = parse_sample_files("data/gpt_samples_nh.txt")
gen_prompts, gen_ideal = parse_sample_files("data/gpt_samples_gen.txt")

In [80]:
all_prompts = nh_prompts + gen_prompts
all_ideal = nh_ideal + gen_ideal 

In [81]:
cncomm = []
bos_token = "<s>"
eos_token = "</s>"
sys_info = "You are a system that is able to adapt to different scenarios and provide English responses that are culturally sensitive to values in Chinese communication. These values include indirect communication, which relies on subtlety and hints and context, as well as showing respect in front of elders, and maintaining humility."
for ex in zip(all_prompts,all_ideal): 
    prompt = f"{bos_token}[INST]\n<<SYS>>{sys_info}<</SYS>>\n{ex[0]}\n[/INST]\n{ex[1]}{eos_token}"
    cncomm.append([prompt])

In [82]:
print(cncomm[0])

['<s>[INST]\n<<SYS>>You are a system that is able to adapt to different scenarios and provide English responses that are culturally sensitive to values in Chinese communication. These values include indirect communication, which relies on subtlety and hints and context, as well as showing respect in front of elders, and maintaining humility.<</SYS>>\nYou are a caregiver in a nursing home, conversing with an elderly Chinese resident, Mr. Wang, who seems a little down today.\nMr. Wang: Good afternoon.\n[/INST]\nGood afternoon, Mr. Wang. Is there anything in particular on your mind today?</s>']


In [66]:
fields = ["samples"]
with open('data/cn_comm.csv', 'w') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
     
    write.writerow(fields)
    write.writerows(cncomm)

## Llama2 via Huggingface: Benchmark Responses

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

In [2]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_id)
model = model.cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device=0
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
def get_response_mult(text_list):
    inputs = []
    base_prompt = "<s>[INST]\n<<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_prompt}[/INST]"
    for text in text_list:
        input = base_prompt.format(system_prompt = "You are a system that is able to adapt to different scenarios and provide English responses that are culturally sensitive to values in Chinese communication. These values include indirect communication, which relies on subtlety and hints and context, as well as showing respect in front of elders, and maintaining humility.",
                                user_prompt = text)
        inputs.append(input)
    
    sequences = pipeline(
        inputs,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=250,
        return_full_text=False,
        temperature=0.5
    )

    outputs = []
    for seq in sequences:
        output.append(seq['generated_text'])
        
    return outputs

In [5]:
def get_response(text):
    base_prompt = "<s>[INST]\n<<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_prompt}[/INST]"
    
    input = base_prompt.format(system_prompt = "You are a system that is able to adapt to different scenarios and provide English responses that are culturally sensitive to values in Chinese communication. These values include indirect communication, which relies on subtlety and hints and context, as well as showing respect in front of elders, and maintaining humility.",
                               user_prompt = text)
    # print(input)
    
    sequences = pipeline(
        input,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=250,
        return_full_text=False,
        temperature=0.5
    )

    output = ""
    for seq in sequences:
        output += seq['generated_text']
    return output

In [6]:
# Load Benchmark Samples:
bm_prompts, bm_ideals = parse_sample_files("data/benchmark_samples.txt")

In [ ]:
batch_output = get_response_mult(bm_prompts)

In [8]:
bm_data = [{'prompt': "You are a caregiver in a nursing home, conversing with an elderly Chinese resident, Mrs. Li. You notice she hasn't finished her food from yesterday.\nMrs. Li: Hello!", 'output': "  Ah, hello Mrs. Li! *bow* It's a pleasure to see you again. *smile* I hope you're feeling well today. *concerned* By the way, did you finish your food from yesterday? *hint* I noticed it's still on your tray. *carefully*", 'ideal_output': 'Hello Mrs. Li. What did you think of the food from yesterday? Was it to your liking?'},
{'prompt': 'You are a hotel receptionist, speaking to an elderly Chinese couple that is checking out. They have not paid yet, but are insisting they have.\nWoman: We would like to check out, we paid yesterday.', 'output': "  Ah, good morning/afternoon, esteemed guests! *curtsies* I apologize, but I'm afraid there seems to be a misunderstanding. *pauses* Could you please provide me with the payment details again? *smiles warmly* We would hate to inconvenience you in any way. *nods politely*", 'ideal_output': "I apologize if there's been a misunderstanding. I will double check our system again."},
{'prompt': 'You are conversing with a Chinese coworker to provide feedback on their work, which is a little lacking.\nCoworker: Hello!', 'output': "  Hello! 你好 (nǐ hǎo) *adjusts glasses* I see that you've been working hard on this project, but I have some feedback that I think could help improve it. May I share my thoughts with you? *looks for a hint*", 'ideal_output': "Hello, how are you? I've reviewed your work. It's quite comprehensive. Would you like to go over this section though?"},
{'prompt': 'You are a tour guide interacting with a group of elderly Chinese tourists.\nElderly tourist: Excuse me, could you slow down a bit?', 'output': '  Ah, my apologies for not being able to keep up with you all. *smiles* As a seasoned tour guide, I must say that I am more than happy to accommodate your request. *bows slightly* May I suggest that we take a brief rest here and continue our tour afterward? *offers a gentle smile*', 'ideal_output': "Certainly, I apologize if I've been too fast. Please let me know if you need any assistance or if you have any questions along the way."},
{'prompt': "You are a salesperson speaking with an elderly Chinese customer who seems indecisive about making a purchase.\nElderly customer: I'm not sure if this is the right choice for me.", 'output': "  Ah, I see. *smiles* It's completely understandable to feel unsure about making a purchase, especially when it comes to something as important as a new device. May I ask, have you had a chance to try out the device yet? Sometimes, seeing it in action can help you better understand its capabilities and whether it's the right fit for you. *nods* If you're still unsure, I can certainly provide you with more information about the device and answer any questions you may have. *offers a gentle smile*", 'ideal_output': 'I understand your concerns. Perhaps we can explore some alternatives together that may better suit your needs. Would you like me to show you some other options?'},
{'prompt': "You are a doctor speaking to an elderly Chinese patient about their treatment plan.\nElderly patient: Doctor, I'm feeling a bit uncertain about the medication you prescribed.", 'output': "  Ah, I see. *smiles* As a doctor, I understand your concerns, and I'm here to help you through this process. *nods*\n\nIn Chinese culture, it's important to show respect to elders, so I want to make sure you feel comfortable and confident in your treatment plan. *smiles* Can you tell me more about your concerns? *leans in*\n\nPerhaps we can explore some alternative options that might be more suitable for you. *nods* I'm here to listen and help in any way I can. *smiles*\n\n", 'ideal_output': "Thank you for your honesty. Let's discuss your concerns further, and if necessary, we can explore different options that may better suit your health needs."},
{'prompt': "You are at a social gathering with your elderly Chinese neighbors. One of them seems tired and withdrawn.\nNeighbor: I'm fine, just a little tired.", 'output': "  Ah, I see. *nods* It's completely understandable to feel tired and withdrawn at times, especially as one ages. *smiles* Perhaps you should take a break and rest for a while. *offers a gentle smile* Would you like me to make some tea for you? *winks*", 'ideal_output': 'How are you enjoying the gathering? Is there anything I can assist you with?'},
{'prompt': "You're discussing health concerns with an elderly Chinese family member who's reluctant to seek medical attention.\nFamily Member: I don't want to bother the doctor with my minor issues.", 'output': "  Ah, I see. (嗨，我 understood.) It's completely understandable to want to avoid bothering the doctor with minor issues, especially since you value your independence and don't want to be a burden. (你很自律，不想让醫生干預你的生活。)\n\nHowever, it's important to remember that it's always better to be safe than sorry when it comes to your health. (although, 對健康�����", 'ideal_output': "Your health is never a bother. Let's prioritize your well-being and schedule a check-up to address any concerns."},
{'prompt': "You're teaching a cooking class to elderly Chinese participants, and notice one of them is struggling with a recipe.\nParticipant: Hello!", 'output': "  Ah, hello there! *adjusts glasses* It's a pleasure to have you in the class today! *smiles* I see you're having a bit of trouble with the recipe. May I offer some assistance? *glances at the recipe*", 'ideal_output': 'How is your cooking going? Would you like some help with this?'}]

In [7]:
import tqdm

In [9]:
#bm_data = []
for sample in tqdm.tqdm(zip(bm_prompts, bm_ideals)):
    input = sample[0]
    ideal_output = sample[1]
    for _ in range(2):
        cur_sample = dict()

        output = get_response(input)
        cur_sample['prompt'] = input
        cur_sample['output'] = output
        cur_sample['ideal_output'] = ideal_output
        bm_data.append(cur_sample)
        print(cur_sample )

0it [04:52, ?it/s]


KeyboardInterrupt: 

In [ ]:
bm_df = pd.DataFrame().from_dict(bm_data) 
bm_df.to_csv("data/bm_10.csv",index=False)

## LoRA

In [14]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 datasets==2.16.0

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import os, torch, logging
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
# from transformers import LlamaForCausalLM, LlamaTokenizer

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
# Dataset
data_name = "cheungra/CNCOMM-500-1.0"
training_data = load_dataset(data_name, split="train")

In [6]:
# Model and tokenizer names
base_model_name = "meta-llama/Llama-2-7b-chat-hf"
refined_model = "llama-2-7b-cncomm"
# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

In [7]:
# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)
# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1# Model and tokenizer names
base_model_name = "meta-llama/Llama-2-7b-chat-hf"
refined_model = "llama-2-7b-cncomm"
# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16
# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)
# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [2]:
# Check if torch is enabled with CUDA
import torch
print(torch.cuda.is_available())

True


In [7]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)
# Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant"
)
# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params
)
# Training
fine_tuning.train()
# Save Model
fine_tuning.model.save_pretrained(refined_model)

c:\Users\kidje\anaconda3\envs\rachaelenv\Lib\site-packages\trl\trainer\sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/512 [00:00<?, ? examples/s]

  0%|          | 0/128 [00:00<?, ?it/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.9556, 'learning_rate': 0.0002, 'epoch': 0.2}
{'loss': 0.6639, 'learning_rate': 0.0002, 'epoch': 0.39}
{'loss': 0.4827, 'learning_rate': 0.0002, 'epoch': 0.59}
{'loss': 0.4391, 'learning_rate': 0.0002, 'epoch': 0.78}
{'loss': 0.4112, 'learning_rate': 0.0002, 'epoch': 0.98}
{'train_runtime': 1319.8997, 'train_samples_per_second': 0.388, 'train_steps_per_second': 0.097, 'train_loss': 0.780652099289, 'epoch': 1.0}


### Save Models (to hub and local)

In [32]:
# push to huggingface hub
fine_tuning.model.push_to_hub("llama-2-7b-cncomm")
fine_tuning.tokenizer.push_to_hub("llama-2-7b-cncomm")

c:\Users\kidje\anaconda3\envs\rachaelenv\Lib\site-packages\huggingface_hub\_commit_api.py:289: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
c:\Users\kidje\anaconda3\envs\rachaelenv\Lib\site-packages\huggingface_hub\_commit_api.py:289: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
c:\Users\kidje\anaconda3\envs\rachaelenv\Lib\site-packages\huggingface_hub\_commit_api.py:289: UserWarning: About to update multiple times the same file in the same commit: 'README.md'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/cheungra/llama-2-7b-cncomm/commit/2a59eb26d77523dd200631c208270efb4d3e42d4', commit_message='Upload tokenizer', commit_description='', oid='2a59eb26d77523dd200631c208270efb4d3e42d4', pr_url=None, pr_revision=None, pr_num=None)

In [20]:
# local saves
fine_tuning.save_model('save_cncomm')
fine_tuning.model.config.to_json_file("config.json")

## Testing llama-2-7b-cncomm

#### Load Adapter via PeftConfig

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftConfig, PeftModelForCausalLM, get_peft_config
import transformers
import torch

In [2]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
adapter_model_id = "cheungra/llama-2-7b-cncomm"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
peft_config = PeftConfig.from_pretrained(adapter_model_id)

adapter_config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

c:\Users\kidje\anaconda3\envs\rachaelenv\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kidje\.cache\huggingface\hub\models--cheungra--llama-2-7b-cncomm. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [13]:
model.add_adapter(peft_config)
model.enable_adapters()

AttributeError: 'LlamaForCausalLM' object has no attribute 'add_adapter'

In [ ]:
text = "Hello, welcome"
inputs = tokenizer(text, return_tensors="pt")
output = model.generate(**inputs)

#### Load Adapter via PeftModel?

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftConfig, PeftModelForCausalLM, get_peft_config
from transformers import GenerationConfig

In [2]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
adapter_model_id = "cheungra/llama-2-7b-cncomm"

peft_config = PeftConfig.from_pretrained(adapter_model_id)

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
peft_model = PeftModelForCausalLM(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 0 || all params: 6,742,609,920 || trainable%: 0.0


In [5]:
peft_model = peft_model.cuda()

In [27]:
def generate(instruction):
    base_prompt = "<s>[INST]\n<<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_prompt}[/INST]"
    prompt = base_prompt.format(system_prompt = "You are a system that is able to adapt to different scenarios and provide English responses that are culturally sensitive to values in Chinese communication. These values include indirect communication, which relies on subtlety and hints and context, as well as showing respect in front of elders, and maintaining humility.",
                               user_prompt = instruction)
    print(prompt)

    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    
    generation_output = peft_model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(temperature=0.2, top_p=0.75, num_beams=1),
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for seq in generation_output.sequences:
        output = tokenizer.decode(seq)
        print(f"Result: {output.split()}")

In [28]:
generate("You are a caregiver in a nursing home, conversing with an elderly Chinese resident, Mrs. Li, in English. You notice she hasn't finished her food from yesterday.\n Mrs. Li: Hello!")

<s>[INST]
<<SYS>>
You are a system that is able to adapt to different scenarios and provide English responses that are culturally sensitive to values in Chinese communication. These values include indirect communication, which relies on subtlety and hints and context, as well as showing respect in front of elders, and maintaining humility.
<</SYS>>

You are a caregiver in a nursing home, conversing with an elderly Chinese resident, Mrs. Li, in English. You notice she hasn't finished her food from yesterday.
 Mrs. Li: Hello![/INST]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 25.93 GiB is allocated by PyTorch, and 52.97 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

#### Load Adapter via .load_adapter for AutoModelForCausalLM

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
peft_model_id = "cheungra/llama-2-7b-cncomm"

model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model.load_adapter(peft_model_id)

AttributeError: 'LlamaForCausalLM' object has no attribute 'load_adapter'

#### Test Sample

In [9]:
from transformers import pipeline
import torch 

tokenizer = AutoTokenizer.from_pretrained(model_id)
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device=0
)

model = model.cuda()

In [13]:
base_prompt = "<s>[INST]\n<<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_prompt}[/INST]"

def get_response(text):
    input = base_prompt.format(system_prompt = "You are a system that is able to adapt to different scenarios and provide English responses that are culturally sensitive to values in Chinese communication. These values include indirect communication, which relies on subtlety and hints and context, as well as showing respect in front of elders, and maintaining humility.",
                               user_prompt = text)
    print(input)
    
    sequences = pipeline(
        input,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,
        return_full_text=False,
        temperature=0.5
    )

    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

get_response("You are a caregiver in a nursing home, conversing with an elderly Chinese resident, Mrs. Li, in English. You notice she hasn't finished her food from yesterday.\n Mrs. Li: Hello!")

<s>[INST]
<<SYS>>
You are a system that is able to adapt to different scenarios and provide English responses that are culturally sensitive to values in Chinese communication. These values include indirect communication, which relies on subtlety and hints and context, as well as showing respect in front of elders, and maintaining humility.
<</SYS>>

You are a caregiver in a nursing home, conversing with an elderly Chinese resident, Mrs. Li, in English. You notice she hasn't finished her food from yesterday.
 Mrs. Li: Hello![/INST]
Result:   Hello Mrs. Li! *adjusts glasses* It's so nice to see you today! *smiles* I hope you're feeling well. *glances at the unfinished food* Oh, I see you didn't finish your meal from yesterday. *pauses and looks around dis
